## DeGirum `nnexpress` Compiler 
In this notebook, we illustrate the main compiler API that compiles an __onnx__ file to an __n2x__ file. The compiler API accepts a path to an onnx file and the device option. Currently, an onnx file can be compiled for CPU and ORCA (DeGirum's AI HW accelerator). To compile for CPU, specify ` device='SW' ` and to compile for ORCA, specify `device='HW'`

After the model is compiled, it can be saved as a file with extension __n2x__ and then run on HW and SW and the results can be compared to make sure that the model has been compiled correctly. The model's performance in terms of frames per second (FPS) can also be estimated. `nnexpress_utilities` contains helper functions to achieve these goals. 



In [1]:
import nnexpress.compiler as n2xc
from nnexpress_utils import run_n2x, compare_n2x_sw_hw, estimate_fps

2022-10-19 17:17:41.576276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-19 17:17:42.046790: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
n2xc.__version__

'0.0.6'

In [5]:
import nnexpress.compiler as n2xc
import os

onnx_directory_str='/home/degirum/DeGirumModels/ONNXModels/'
n2xsw_directory_str='/home/degirum/DeGirumModels/N2XSWModels/'
n2xhw_directory_str='/home/degirum/DeGirumModels/N2XHWModels/'

onnx_directory = os.fsencode(onnx_directory_str)
n2xsw_directory = os.fsencode(n2xsw_directory_str)
n2xhw_directory = os.fsencode(n2xhw_directory_str)

file_number=0    
for file in os.listdir(onnx_directory):
    filename = os.fsdecode(file)
    if filename.endswith(".onnx"): 
        print(file_number)
        file_number+=1
        tflite_path=os.path.join(onnx_directory, file)
        n2x_sw_path = os.path.join(n2xsw_directory, os.fsencode(filename.split('.')[0]+"_sw.n2x"))
        n2x_hw_path = os.path.join(n2xhw_directory, os.fsencode(filename.split('.')[0]+"_hw.n2x"))
        print('Compiling ',filename)
        sw_model=n2xc.Compiler(tflite_path, device='SW')
        hw_model=n2xc.Compiler(tflite_path, device='HW')
        sw_model.save(n2x_sw_path)
        hw_model.save(n2x_hw_path)


0
Compiling  mobilenet_v1_1.0_224.onnx
1
Compiling  tv_resnet50.onnx
2
Compiling  tv_resnet50_quantized.onnx
3
Compiling  hand_yolov5s_relu6_512_v2_quantized.onnx
4
Compiling  lpr_256_yolov5s_relu6_512_v2.onnx
5
Compiling  efficientnet_es_quantized.onnx
6
Compiling  mobilenetv2_100.onnx
7
Compiling  lpr_256_yolov5s_relu6_512_v2_quantized.onnx
8
Compiling  efficientnet_es.onnx


In [7]:
import os

onnx_directory_str='/home/degirum/DeGirumModels/ONNXModels/'
n2xsw_directory_str='/home/degirum/DeGirumModels/N2XSWModels/'
n2xhw_directory_str='/home/degirum/DeGirumModels/N2XHWModels/'

onnx_directory = os.fsencode(onnx_directory_str)
n2xsw_directory = os.fsencode(n2xsw_directory_str)
n2xhw_directory = os.fsencode(n2xhw_directory_str)

file_number=0    
for i in range(5):
    print(i)
    for file in os.listdir(onnx_directory):
        filename = os.fsdecode(file)
        if filename.endswith(".onnx"): 
            print(file_number)
            file_number+=1
            tflite_path=os.path.join(onnx_directory, file)
            n2x_sw_path = os.path.join(n2xsw_directory, os.fsencode(filename.split('.')[0]+"_sw.n2x"))
            n2x_hw_path = os.path.join(n2xhw_directory, os.fsencode(filename.split('.')[0]+"_hw.n2x"))
            print('Compiling ',filename)
            sw_model=n2xc.Compiler(tflite_path, device='SW')
            hw_model=n2xc.Compiler(tflite_path, device='HW')
            sw_model.save(n2x_sw_path)
            hw_model.save(n2x_hw_path)
            print('Running SW n2x file:',run_n2x(n2x_sw_path))
            print('Running HW n2x file:',run_n2x(n2x_hw_path))
            match=compare_n2x_sw_hw(n2x_sw_path,n2x_hw_path)
            print('Comparing SW vs HW\n')
            if match:
                print('N2X SW and HW match')
            else:
                print('N2X SW and HW do not match')
            fps=estimate_fps(n2x_hw_path)
            print('FPS=',fps)
            print('**********************************************\n')


0
0
Compiling  mobilenet_v1_1.0_224.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 66.13827454322501
**********************************************

1
Compiling  tv_resnet50.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 10.958684260968965
**********************************************

2
Compiling  tv_resnet50_quantized.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 40.41442553976301
**********************************************

3
Compiling  hand_yolov5s_relu6_512_v2_quantized.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 59.736670017907784
**********************************************

4
Compiling  lpr_256_yolov5s_relu6_512_v2.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 33.3910

FPS= 40.373737100419405
**********************************************

39
Compiling  hand_yolov5s_relu6_512_v2_quantized.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 58.67288295378673
**********************************************

40
Compiling  lpr_256_yolov5s_relu6_512_v2.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 33.45879923545787
**********************************************

41
Compiling  efficientnet_es_quantized.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 110.2150283962077
**********************************************

42
Compiling  mobilenetv2_100.onnx
Running SW n2x file: success
Running HW n2x file: success
Comparing SW vs HW

N2X SW and HW match
FPS= 75.20085652452046
**********************************************

43
Compiling  lpr_256_yolov5s_relu6_512_v2_quantized.onnx
Running SW 

In [12]:
onnx_path='/home/degirum/DeGirumModels/ONNXModels/tv_resnet50_quantized_hw.onnx'
n2x_sw_path='/home/degirum/DeGirumModels/N2XSWModels/tv_resnet50_quantized_sw.n2x'
n2x_hw_path='/home/degirum/DeGirumModels/N2XHWModels/tv_resnet50_quantized_hw.n2x'
sw_model=n2xc.Compiler(onnx_path, device='SW')
hw_model=n2xc.Compiler(onnx_path, device='HW')
sw_model.save(n2x_sw_path)
hw_model.save(n2x_hw_path)

In [4]:
print(run_n2x(n2x_sw_path))

success


**Note**: This below cells can run only on a device equipped with ORCA

In [5]:
print(run_n2x(n2x_hw_path))

success


In [6]:
match=compare_n2x_sw_hw(n2x_sw_path,n2x_hw_path)
if match:
    print('N2X SW and HW match')
else:
    print('N2X SW and HW do not match')

N2X SW and HW match


In [7]:
fps=estimate_fps(n2x_hw_path)
print('FPS=',fps)

FPS= 59.61617954210848
